<a href="https://colab.research.google.com/github/vitoriapmaattos/dataanalytics/blob/main/Fase3_Aula1_Conhecendo_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução ao PySpark




In [2]:
#Instalando o PySpark
!pip install pyspark

In [5]:
#Instalando a  FindSpark
!pip install findspark

In [6]:
import findspark #importando o findspark
findspark.init() #iniciando o findspark
from pyspark.sql import SparkSession #importando o SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #criando uma sessão Spark


In [10]:
#Testando a utilização do spark sql
df = spark.sql('''select 'spark' as hello''') #criando um dataframe com o nome spark
df.show() #mostrando o dataframe

+-----+
|hello|
+-----+
|spark|
+-----+



In [12]:
#Impotando as bibliotecas do Spark
from pyspark.sql import Row, DataFrame #importando as classes Row e DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType #importando os tipos de dados
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce #importando as funções do pyspark
from pyspark.sql import functions as F #para mais funções do sql
from functools import reduce #para juntar dataframes

# Data Manipulation usando Spark

In [13]:
#importando a biblioteca
df = spark.read.csv('banklist.csv', sep=',', header=True, inferSchema=True) #importando o arquivo csv

print('df.count :', df.count()) #mostrando a quantidade de linhas do dataframe
print('df.col ct :', len(df.columns)) #mostrando a quantidae de colunas do dataframe
print('df.columns :', df.columns) #mostrando as colunas do dataframe

df.count : 561
df.col ct : 6
df.columns : ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


# Using SQL in PySpark

In [16]:
df.createOrReplaceTempView('banklist') #criando uma view temporária do dataframe

df_check = spark.sql(''' select `Bank Name`, `City`, `Closing Date` from banklist''') #criando um dataframe com a view temporária
df_check.show() #mostrando o dataframe

+--------------------+------------------+------------+
|           Bank Name|              City|Closing Date|
+--------------------+------------------+------------+
|The First State Bank|     Barboursville|    3-Apr-20|
|  Ericson State Bank|           Ericson|   14-Feb-20|
|City National Ban...|            Newark|    1-Nov-19|
|       Resolute Bank|            Maumee|   25-Oct-19|
|Louisa Community ...|            Louisa|   25-Oct-19|
|The Enloe State Bank|            Cooper|   31-May-19|
|Washington Federa...|           Chicago|   15-Dec-17|
|The Farmers and M...|           Argonia|   13-Oct-17|
| Fayette County Bank|        Saint Elmo|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee|    5-May-17|
|      First NBC Bank|       New Orleans|   28-Apr-17|
|       Proficio Bank|Cottonwood Heights|    3-Mar-17|
|Seaway Bank and T...|           Chicago|   27-Jan-17|
|Harvest Community...|        Pennsville|   13-Jan-17|
|         Allied Bank|          Mulberry|   23-Sep-16|
|The Woodb

# Operações Básicas de DataFrame

In [17]:
df.describe().show() #mostrando as estatísticas do dataframe

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [18]:
df.describe('City', 'ST').show() #mostrando as estatísticas da coluna City e State

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



In [22]:
#Count, Columns e Schema

print('Total de Linhas :', df.count()) #mostrando a quantidade de linhas do dataframe
print('Total de Colunas :', len(df.columns)) #mostrando a quantidae de colunas do dataframe
print('Colunas :', df.columns) #mostrando as colunas do dataframe
print('Schema: ', df.schema) #mostrando o schema do dataframe
print('Tipo de Dados :', df.dtypes) #mostrando os tipos de dados das colunas

Total de Linhas : 561
Total de Colunas : 6
Colunas : ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Schema:  StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])
Tipo de Dados : [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]


In [23]:
df.printSchema() #mostrando o schema do dataframe

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



In [27]:
#Removendo duplicados

df.dropDuplicates() #removendo os duplicados do dataframe
print('Total de Linhas :', df.count()) #mostrando a quantidade de linhas do dataframe
print('Colunas :', df.columns) #mostrando as colunas do dataframe

Total de Linhas : 561
Colunas : ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


In [30]:
#Selecionando colunas específicas - FUNCTION SELECT

df2 = df.select(*['Bank Name', 'City']) #selecionando as colunas Bank Name e City
df2.show(2) #mostrando o dataframe com 2 linhas


+--------------------+-------------+
|           Bank Name|         City|
+--------------------+-------------+
|The First State Bank|Barboursville|
|  Ericson State Bank|      Ericson|
+--------------------+-------------+
only showing top 2 rows



In [32]:
#Selecionando múltiplas colunas
col_1 = list(set(df.columns) - {'CERT', 'ST'}) #selecionando as colunas para não aparecer
df2 = df.select(*col_1) #selecionando as colunas
df2.show(2) #mostrando o dataframe com 2 linhas

+------------+-------------+--------------------+---------------------+
|Closing Date|         City|           Bank Name|Acquiring Institution|
+------------+-------------+--------------------+---------------------+
|    3-Apr-20|Barboursville|The First State Bank|       MVB Bank, Inc.|
|   14-Feb-20|      Ericson|  Ericson State Bank| Farmers and Merch...|
+------------+-------------+--------------------+---------------------+
only showing top 2 rows



In [33]:
#Renomear colunas

df2 = df \
  .withColumnRenamed('Bank Name', 'bank_name') \
  .withColumnRenamed('Acquiring Institution', 'acq_institution') \
  .withColumnRenamed('Closing Date', 'closing_date') \
  .withColumnRenamed('ST', 'state') \
  .withColumnRenamed('CERT', 'cert')
df2.show(2) #mostrando o dataframe com 2 linhas

+--------------------+-------------+-----+-----+--------------------+------------+
|           bank_name|         City|state| cert|     acq_institution|closing_date|
+--------------------+-------------+-----+-----+--------------------+------------+
|The First State Bank|Barboursville|   WV|14361|      MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson|   NE|18265|Farmers and Merch...|   14-Feb-20|
+--------------------+-------------+-----+-----+--------------------+------------+
only showing top 2 rows



In [34]:
#Adicionando colunas

df2 = df.withColumn('state', col('ST')) #adicionando a coluna state com os dados da coluna ST
df2.show() #mostrando o dataframe

+--------------------+------------------+---+-----+---------------------+------------+-----+
|           Bank Name|              City| ST| CERT|Acquiring Institution|Closing Date|state|
+--------------------+------------------+---+-----+---------------------+------------+-----+
|The First State Bank|     Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|   WV|
|  Ericson State Bank|           Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|   NE|
|City National Ban...|            Newark| NJ|21111|      Industrial Bank|    1-Nov-19|   NJ|
|       Resolute Bank|            Maumee| OH|58317|   Buckeye State Bank|   25-Oct-19|   OH|
|Louisa Community ...|            Louisa| KY|58112| Kentucky Farmers ...|   25-Oct-19|   KY|
|The Enloe State Bank|            Cooper| TX|10716|   Legend Bank, N. A.|   31-May-19|   TX|
|Washington Federa...|           Chicago| IL|30570|   Royal Savings Bank|   15-Dec-17|   IL|
|The Farmers and M...|           Argonia| KS|17719|          Conway Ba

In [35]:
#Adicionando coluna constante

df2 = df.withColumn('country', lit('US')) #adicionando a coluna country com dados fixos (US)
df2.show() #mostrando o dataframe

+--------------------+------------------+---+-----+---------------------+------------+-------+
|           Bank Name|              City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+------------------+---+-----+---------------------+------------+-------+
|The First State Bank|     Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|     US|
|  Ericson State Bank|           Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|     US|
|City National Ban...|            Newark| NJ|21111|      Industrial Bank|    1-Nov-19|     US|
|       Resolute Bank|            Maumee| OH|58317|   Buckeye State Bank|   25-Oct-19|     US|
|Louisa Community ...|            Louisa| KY|58112| Kentucky Farmers ...|   25-Oct-19|     US|
|The Enloe State Bank|            Cooper| TX|10716|   Legend Bank, N. A.|   31-May-19|     US|
|Washington Federa...|           Chicago| IL|30570|   Royal Savings Bank|   15-Dec-17|     US|
|The Farmers and M...|           Argonia| KS|17719

In [36]:
#Apagando colunas - DROP

df2 = df.drop('CERT') #apagando a coluna CERT
df2.show() #mostrando o dataframe

+--------------------+------------------+---+---------------------+------------+
|           Bank Name|              City| ST|Acquiring Institution|Closing Date|
+--------------------+------------------+---+---------------------+------------+
|The First State Bank|     Barboursville| WV|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|           Ericson| NE| Farmers and Merch...|   14-Feb-20|
|City National Ban...|            Newark| NJ|      Industrial Bank|    1-Nov-19|
|       Resolute Bank|            Maumee| OH|   Buckeye State Bank|   25-Oct-19|
|Louisa Community ...|            Louisa| KY| Kentucky Farmers ...|   25-Oct-19|
|The Enloe State Bank|            Cooper| TX|   Legend Bank, N. A.|   31-May-19|
|Washington Federa...|           Chicago| IL|   Royal Savings Bank|   15-Dec-17|
|The Farmers and M...|           Argonia| KS|          Conway Bank|   13-Oct-17|
| Fayette County Bank|        Saint Elmo| IL| United Fidelity B...|   26-May-17|
|Guaranty Bank, (d...|      

In [38]:
#Apagando múltiplas colunas - DROP

df2 = df.drop(*['CERT', 'ST']) #apagando as colunas CERT e ST
df2.show() #mostrando o dataframe

+--------------------+------------------+---------------------+------------+
|           Bank Name|              City|Acquiring Institution|Closing Date|
+--------------------+------------------+---------------------+------------+
|The First State Bank|     Barboursville|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|           Ericson| Farmers and Merch...|   14-Feb-20|
|City National Ban...|            Newark|      Industrial Bank|    1-Nov-19|
|       Resolute Bank|            Maumee|   Buckeye State Bank|   25-Oct-19|
|Louisa Community ...|            Louisa| Kentucky Farmers ...|   25-Oct-19|
|The Enloe State Bank|            Cooper|   Legend Bank, N. A.|   31-May-19|
|Washington Federa...|           Chicago|   Royal Savings Bank|   15-Dec-17|
|The Farmers and M...|           Argonia|          Conway Bank|   13-Oct-17|
| Fayette County Bank|        Saint Elmo| United Fidelity B...|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee| First-Citizens Ba...|    5-May-17|

In [39]:
#Apagando múltiplas colunas - REDUCE

df2 = reduce(DataFrame.drop, ['CERT', 'ST'], df) #apagando as colunas CERT e ST
df2.show() #mostrando o dataframe

+--------------------+------------------+---------------------+------------+
|           Bank Name|              City|Acquiring Institution|Closing Date|
+--------------------+------------------+---------------------+------------+
|The First State Bank|     Barboursville|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|           Ericson| Farmers and Merch...|   14-Feb-20|
|City National Ban...|            Newark|      Industrial Bank|    1-Nov-19|
|       Resolute Bank|            Maumee|   Buckeye State Bank|   25-Oct-19|
|Louisa Community ...|            Louisa| Kentucky Farmers ...|   25-Oct-19|
|The Enloe State Bank|            Cooper|   Legend Bank, N. A.|   31-May-19|
|Washington Federa...|           Chicago|   Royal Savings Bank|   15-Dec-17|
|The Farmers and M...|           Argonia|          Conway Bank|   13-Oct-17|
| Fayette County Bank|        Saint Elmo| United Fidelity B...|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee| First-Citizens Ba...|    5-May-17|

In [42]:
#Filtrando valores dentro do DataFrame

#filtro = um valor
df2 = df.where(df['ST'] == 'NE') #filtrando os valores da coluna ST que são iguais a NE

#filtro entre dois valores
df3 = df.where(df['CERT'].between(1000, 2000)) #filtrando os valores da coluna CERT que estão entre 1000 e 2000

#filtro = mais de um valor
df4 = df.where(df['ST'].isin('NE', 'IL')) #filtrando os valores da coluna ST que são iguais a NE e IL

print('df2.count :', df2.count()) #mostrando a quantidade de linhas do dataframe
print('df3.count :', df3.count()) #mostrando a quantidade de linhas do dataframe
print('df4.count :', df4.count()) #mostrando a quantidade de linhas do dataframe

df2.count : 4
df3.count : 9
df4.count : 73


In [43]:
#Filtrando usando operadores lógicos

df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson')) #filtrando os valores da coluna ST que são iguais a NE e os valores da coluna City que são iguais a Ericson
df2.show() #mostrando o dataframe

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



In [45]:
#Substituindo valores
df.show(2) #mostrando o dataframe
print('Replace 7 in the above dataframe with 17 at all instances')
df.na.replace(7, 17).show(2) #substituindo os valores 7 por 17



+--------------------+-------------+---+-----+---------------------+------------+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+---+-----+---------------------+------------+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+--------------------+-------------+---+-----+---------------------+------------+
only showing top 2 rows

Replace 7 in the above dataframe with 17 at all instances
+--------------------+-------------+---+-----+---------------------+------------+
|           Bank Name|         City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+-------------+---+-----+---------------------+------------+
|The First State Bank|Barboursville| WV|14361|       MVB Bank, Inc.|    3-Apr-20|
|  Ericson State Bank|      Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+--------------